## Libraries

In [4]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import psycopg2 as pg2
import matplotlib.pyplot as plt 
import seaborn as sns

## Read CSV

In [5]:
import glob 

directory = "data"

all_files=glob.glob(directory + "/*.csv")
file_dict = {}

def import_files():
    for file_name in all_files:
        df = pd.read_csv(file_name)
        file_dict[file_name] = df

# Data Analysis

In [7]:
database = "fish2"
host = "wellwoodbdev"
user = "user9"
port = "5432"
schema = "age_user9"
table = "t_age"
white_space = " "
query = "select * from {}.%s LIMIT 0".format(schema)
conn_str = "dbname={} host={} user={} port={}".format(database, host, user, port)

def get_all_table_names():
    tables = []
    try:
        with pg2.connect(conn_str) as conn:
            with conn.cursor() as curs:
                curs.execute("""
                SELECT table_name FROM information_schema.tables
                   WHERE table_schema = 'age_user9'
                """)
                for (table,) in curs.fetchall():
                    tables.append(table)
    except (Error) as error:
        print("Error {}".format(error))
    finally:
        return tables
  
def remove_whitespace():
    for key,file in file_dict.items():
        file.columns = file.columns.str.strip().str.lower().str.replace(' ', '')
        
def check_column_names(table, file):
    print("\n\nTable: {}\n".format(table)) 
    actual = "none"
    if len(file.columns) == 0:
        print ("Empty data frame")
    try:
        with pg2.connect(conn_str) as conn:
            with conn.cursor() as curs:
                curs.execute(query % table)
                expected_names = [desc[0] for desc in curs.description]
                
                for expected in expected_names:
                    if not(expected in file.columns):
                        actual = "none"
                        for a in file.columns:
                            if expected in a:
                                actual = a
                        if actual == 'none':
                            print("Missing column from table: {}, Expected: '{}'".format(table, expected))
                        else: 
                            print("Incorrecly names column, Expected: '{}', Actual: '{}'".format(expected, actual))
    except (Error) as error:
        print("Error {}".format(error))
    finally:
        if actual == "none":
            print("Complete")

def check_all_columns():
    col_names = get_all_table_names()
    print("Tables:")
    print(col_names)
    table = ''
    for key,file in file_dict.items():
        for c in col_names:
            if c in key:
                table = c
        check_column_names(table, file)       
        
def summary():
    print("Database: {}".format(database))
    print("Host: {}".format(host))
    print("User: {}".format(user))
    print("port: {}".format(port))
    print("\n")

    

In [8]:
summary()

import_files()

check_all_columns()

print("Initial Comparison\n\n")
import_files()

print("Remove Whitespaces\n\n")
remove_whitespace()

print("Second Comparison\n\n")
check_all_columns()

Database: fish2
Host: wellwoodbdev
User: user9
port: 5432


Tables:
['t_age', 't_age_meth', 't_catalog', 't_fish', 't_material', 't_origin', 't_reader', 't_reading', 't_trip']


Table: t_age

Incorrecly names column, Expected: 'yr', Actual: '    yr'
Incorrecly names column, Expected: 'sample_no', Actual: 'sub_sample_no'
Incorrecly names column, Expected: 'area', Actual: 'area '
Incorrecly names column, Expected: 'fish_no', Actual: '    fish_no'
Incorrecly names column, Expected: 'prep_no', Actual: 'prep_no   '
Incorrecly names column, Expected: 'proj_code', Actual: 'proj_code   '


Table: t_catalog

Incorrecly names column, Expected: 'yr', Actual: '    yr'
Incorrecly names column, Expected: 'sample_no', Actual: 'sub_sample_no'
Incorrecly names column, Expected: 'area', Actual: 'area '
Incorrecly names column, Expected: 'fish_no', Actual: '    fish_no'
Incorrecly names column, Expected: 'room', Actual: 'room                                              '
Incorrecly names column, Expecte